In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import binary_crossentropy,mean_squared_error, categorical_crossentropy
from keras.optimizers import SGD, adam
from keras.datasets import mnist
from keras.utils import plot_model,to_categorical
from keras.callbacks import Callback
from keras.models import load_model
from keras import backend as K
import tensorflow as tf
from tensorflow.python.framework.graph_util import convert_variables_to_constants
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [42]:
def get_name(partial):
    for i in df_labels.index:
        if partial in i:
            return i

In [47]:
df = pd.read_csv("/home/fvalle/phd/datasets/tcga/oversampling_10tissue/topsbm/topsbm_level_2_topic-dist.csv", index_col=1).drop('i_doc', axis=1).dropna(how='any', axis=0)
df_labels=pd.read_csv("/home/fvalle/phd/datasets/tcga/oversampling_10tissue/files.dat", index_col=0).dropna(how='any', axis=0)
#df=df.reindex(index=df_labels.index)
df.index=[get_name(name) for name in df.index]

In [49]:
df_labels=df_labels.reindex(df.index).fillna(0)

In [50]:
uniq = len(df_labels['primary_site'].unique())
inputs = df.shape[1]

In [61]:
X_train, X_test, Y_train, Y_test = train_test_split(df.values, to_categorical(np.unique(df_labels['primary_site'], return_inverse=True)[1]), test_size=0.2)

In [62]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [74]:
K.clear_session()
model=Sequential()
#model.add(Dense(units=9, input_dim=inputs, use_bias=True, bias_initializer='ones', activation="relu"))
model.add(Dense(units=uniq, input_dim=inputs, activation="softmax"))
model.compile(loss=categorical_crossentropy, optimizer=adam(lr=0.9), metrics=['accuracy', auc])
plot_model(model, show_shapes=True)

In [75]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 13)                23933     
Total params: 23,933
Trainable params: 23,933
Non-trainable params: 0
_________________________________________________________________


In [76]:
K.set_learning_phase(0)
#model.save("model.h5")
#model=load_model("model.h5")

In [77]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in
                                  tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                          output_names, freeze_var_names)
    return frozen_graph

In [78]:
model.fit(X_train, Y_train, epochs=50, verbose=1, validation_split=0.25)

Train on 475 samples, validate on 159 samples
Epoch 1/50
475/475 [==============================] - 0s 824us/step - loss: 2.3648 - acc: 0.1642 - auc: 0.6252 - val_loss: 2.2252 - val_acc: 0.4025 - val_auc: 0.7042
Epoch 2/50
475/475 [==============================] - 0s 100us/step - loss: 1.6829 - acc: 0.5874 - auc: 0.7606 - val_loss: 1.6625 - val_acc: 0.7421 - val_auc: 0.8134
Epoch 3/50
475/475 [==============================] - 0s 74us/step - loss: 1.1649 - acc: 0.9032 - auc: 0.8521 - val_loss: 1.3257 - val_acc: 0.8113 - val_auc: 0.8800
Epoch 4/50
475/475 [==============================] - 0s 105us/step - loss: 0.8705 - acc: 0.9305 - auc: 0.8999 - val_loss: 1.0916 - val_acc: 0.8994 - val_auc: 0.9152
Epoch 5/50
475/475 [==============================] - 0s 71us/step - loss: 0.6928 - acc: 0.9389 - auc: 0.9267 - val_loss: 0.9150 - val_acc: 0.8994 - val_auc: 0.9358
Epoch 6/50
475/475 [==============================] - 0s 87us/step - loss: 0.5465 - acc: 0.9684 - auc: 0.9431 - val_loss: 0.77

475/475 [==============================] - 0s 77us/step - loss: 0.0227 - acc: 1.0000 - auc: 0.9953 - val_loss: 0.3280 - val_acc: 0.9686 - val_auc: 0.9953


In [82]:
model.evaluate(X_test, Y_test)

159/159 [==============================] - 0s 61us/step


[0.1179296114722138, 0.9685534534964172, 0.9953180697729003]

In [83]:
frozen_graph = freeze_session(K.get_session(),output_names=[out.op.name for out in model.outputs])

In [84]:
tf.train.write_graph(frozen_graph, "TF_model/", "tf_model.pb", as_text=False)

'TF_model/tf_model.pb'

## optimize
```python mo_tf.py --data_type FP16 --input_model /Users/filippo/Developer/TF_model/tf_model.pb --input_shape "[1,409]" --model_name sequential_1 --input dense_1_input --output dense_1/Softmax```

In [ ]:
pd.DataFrame(data=X_test).to_csv("input.txt", index=False, header=False, sep='\t')
pd.DataFrame(data=Y_test.argmax(axis=1)).astype(int).to_csv("test.txt", index=False, header=False, sep='\t')

In [ ]:
np.genfromtxt("test.txt", dtype=int)

In [ ]:
model.predict(X_test[:10]).argmax(axis=1)

In [ ]:
Y_test.argmax(axis=1)[:10]

In [ ]:
np.unique(df_labels['primary_site'])

In [ ]:
X_test